In [3]:
# Imports from python libraries
import numpy as np
import os
import pandas as pd
import string
import sys
from whoosh.index import open_dir

# Imports from own script
from baseline_search import create_searchable_data2
from baseline_search import copy_fragments

# Define paths
root = os.path.join(os.sep,"media","sf_MartinedeVos")
search_dir = os.path.join(os.sep,root,"TargetSize150","text_preserve_paragraph")
indexdir = os.path.join(os.sep,search_dir,"indexdir")
manual_path = os.path.join(root,"TargetSize150","Manual_annotation_sets-20190110T093859Z-001")

**1. Select folder with text fragments in the zip folder on surfdrive:**

../Data/NR-teksts/EviDENce_NR_output/TargetSize100/Lemma_preserve_paragraph.zip

*NB: The file names are long, and so is the path. Make sure to extract the zip folder on high-level location on your computer to avoid "path-too-long" error*

**2. Index all documents (i.e., lemma fragments) in the directory**

* Create Schema
* Add documents
* Perform indexing

_NB: this step only has to be run once, or when data is added or changed_

In [4]:
# The creation of an index is only needed once; after that, opening the existing index is sufficient
# in that case, the following line should be commented out

#create_searchable_data2(search_dir)

**3. Store required data for manual annotation **

* Store data of all fragments in corpus in a dataframe
* Take a random sample
* Store the sample, i.e., title+text, in a csv file

In [3]:
ix = open_dir(indexdir)

with ix.searcher() as searcher:
    index_dic = {doc['title']:[doc['textdata']] for doc in searcher.all_stored_fields()}   

index_df = pd.DataFrame.from_dict(index_dic, orient='index')   

In [4]:
random_state = 1111
sample = index_df.sample(n=100, random_state=random_state)

**Store file set used for manual annotation**

* Based on the sample created above, i.e., csv file with for every fragment title+text
* Store for every textfragment _mentioned_ in the csv file the corresponding _actual_ lemma-based fragment in a separate folder
* The automatic analyses, i.e., keyword search and machine learning, can be performed on the fragments in that folder

In [6]:
source = os.path.join(os.sep,root,"TargetSize150","lemma_preserve_paragraph")
dest = os.path.join(manual_path,"Auto_annotation_sets")

for item in os.listdir(manual_path):
    path = os.path.join(manual_path,item)
    if os.path.isfile(path):
        copy_fragments(path,source, dest)

In [22]:
all_titles = []

for item in os.listdir(manual_path):
    path = os.path.join(manual_path,item)
    if os.path.isfile(path):
        file_df = pd.read_csv(path, sep=',', encoding = "ISO-8859-1").dropna(how='all')
        titles = file_df['Titel'].iloc[:-1].tolist()
        all_titles.append(titles)

manual_set = [title for sublist in all_titles for title in sublist]